# Transformer Base ChatBot From Scratch

## Import Libraries

In [3]:
import numpy
import torch
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'


## Loading DataSet

In [4]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txBlock

--2024-04-07 17:37:19--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txBlock
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-04-07 17:37:19 ERROR 404: Not Found.



In [5]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
print("how mant charachters we have: ", len(text))

how mant charachters we have:  1115394


In [7]:
## Inspect the first 500 characters
print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


### Getting Vocabulary Size
### Get all the unique characters in text


In [8]:
## first lets get number of characters by making a set
numberofchars = sorted(list(set(text)))
vocab_size = len(numberofchars)
print(''.join(numberofchars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


### Tokenizung the characters

In [9]:
dict1 = {c:i for i, c in enumerate(numberofchars)}
dict2 = {i:c for i, c in enumerate(numberofchars)}

# for encoding we take a string and return its tokenize interger version
encoding = lambda Originalstring: [dict1[character] for character in Originalstring ]

# for decoding we take a token sequence and return the characters
decoding = lambda OriginalToken: ''.join([dict2 [token] for token in OriginalToken])

In [10]:
#Testing
print (encoding ("hello world"))
print (decoding (encoding ("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


## Encoding the entire text

In [11]:
allData = torch.tensor(encoding(text),  dtype=torch.long)

In [12]:
## Lets examine the data
print(allData.shape)
print(allData[:10])


torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


## Splitting The Data into Test and Training. For Transformers we typically only keep 10% for testing

In [13]:
training_data = allData[:int(0.9*len(allData))]
testing_data  = allData[int(0.9*len(allData)):]

In [14]:
# this determines the size of the context for each prediction
block = 8

x = training_data [:block]
y = training_data [1:block+1]

print("x is : ", x)
print ("y is : ", y)



x is :  tensor([18, 47, 56, 57, 58,  1, 15, 47])
y is :  tensor([47, 56, 57, 58,  1, 15, 47, 58])


## we wants to see if we give an input of x_1, x_2, ..x_n what will be the expected output after each input sequence

In [15]:
for i in range(block):
    inputs = x[:i+1]
    outputs = y[i]
    print(f"when the input is {inputs} the expected output is {outputs}")

when the input is tensor([18]) the expected output is 47
when the input is tensor([18, 47]) the expected output is 56
when the input is tensor([18, 47, 56]) the expected output is 57
when the input is tensor([18, 47, 56, 57]) the expected output is 58
when the input is tensor([18, 47, 56, 57, 58]) the expected output is 1
when the input is tensor([18, 47, 56, 57, 58,  1]) the expected output is 15
when the input is tensor([18, 47, 56, 57, 58,  1, 15]) the expected output is 47
when the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the expected output is 58


## Batch maker funcion

In [16]:
torch.randint(100-8,(5,))

tensor([76, 50, 46, 57, 82])

### what we are doing in the next function:
### 1. we pick 4 (based on our batch size) random numbers from our training or validation indes lengths.  
### 2. We will then define x as a stack of 4 batches each having 8 cols starting from first random number to random number + 8(size of block)
### 3. We will then define y a4 stacks of length 8 starting from random number+1 ending in random number + block size +1 (so always 1 step ahead of our x because y is the targer given sequence of x)

In [17]:
#this gives us how many sequences of 8 length we wants to process simu
batch= 4


def batchMaker (splitType, batch, block):


    if splitType == 'training':
        data = training_data
    else:
        data = testing_data
    randomcols = torch.randint(len(data) - block , (batch,))
    x = torch.stack ([data[i:i+block] for i in randomcols])
    y = torch.stack ([data[i+1:i+block+1]for i in randomcols])
    x, y = x.to(device), y.to(device)

    return x , y

In [18]:
## testing our function

x, y = batchMaker('training', batch, block)
print ("our input is: \n", x)
print ("our output is: \n", y )


our input is: 
 tensor([[13, 52, 42,  1, 39, 50, 50,  1],
        [25, 13, 30, 21, 13, 26, 13, 10],
        [ 1, 51, 63,  1, 24, 53, 56, 42],
        [39,  1, 46, 59, 52, 42, 56, 43]], device='cuda:0')
our output is: 
 tensor([[52, 42,  1, 39, 50, 50,  1, 58],
        [13, 30, 21, 13, 26, 13, 10,  0],
        [51, 63,  1, 24, 53, 56, 42,  1],
        [ 1, 46, 59, 52, 42, 56, 43, 42]], device='cuda:0')


In [19]:
for j in range(batch):
    for i in range(block):
        inputs = x[j,:i+1]
        outputs = y[j,i]
        print(f"for the input sequence {inputs}, our expected output is {outputs}")

for the input sequence tensor([13], device='cuda:0'), our expected output is 52
for the input sequence tensor([13, 52], device='cuda:0'), our expected output is 42
for the input sequence tensor([13, 52, 42], device='cuda:0'), our expected output is 1
for the input sequence tensor([13, 52, 42,  1], device='cuda:0'), our expected output is 39
for the input sequence tensor([13, 52, 42,  1, 39], device='cuda:0'), our expected output is 50
for the input sequence tensor([13, 52, 42,  1, 39, 50], device='cuda:0'), our expected output is 50
for the input sequence tensor([13, 52, 42,  1, 39, 50, 50], device='cuda:0'), our expected output is 1
for the input sequence tensor([13, 52, 42,  1, 39, 50, 50,  1], device='cuda:0'), our expected output is 58
for the input sequence tensor([25], device='cuda:0'), our expected output is 13
for the input sequence tensor([25, 13], device='cuda:0'), our expected output is 30
for the input sequence tensor([25, 13, 30], device='cuda:0'), our expected output is 2

## Make a Bigram Model

In [20]:
class BigramModel (nn.Module):

    def __init__(self, vocab_size):

        super().__init__()

        #make an embedding tabel size vocab size x vocab size (65 x 65) in this cqw3
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward (self, inputx, targets = None):

        # table will be the token embedding of size (batch, block size , vocab size)
        # (4 x 8 x 65)

        predictions = self.token_embedding_table (inputx)

        # if we dont have targets then no loss
        if targets is None:
            loss = None

        else:
            BatchSize, BlockSize, VocabSize = predictions.shape

            #predictions are in (batch x block x vocab size) dimention
            # cross entropy wants (batch x vocab size x block )

            predictions = predictions.view(BatchSize *BlockSize, VocabSize)
            targets = targets.view(BatchSize *BlockSize)

            # loss is the cross entropy of the predictions and targets

            loss = F.cross_entropy(predictions, targets)

        return predictions, loss

    def generate(self, inputx, max):
        # inputx is (Batch x vocan size) array
        for _ in range(max):
            # get the predictions
            predictions, loss = self(inputx)
            # focus only on the last time step
            predictions = predictions[:, -1, :] # becomes (Batch, vocab size)
            # get probabliries via softmax
            probs = F.softmax(predictions, dim=-1) # (Batch, vocab size)

            # sample from the distribution
            indext_next = torch.multinomial(probs, num_samples=1) # (Batch, 1)

            # concatinate the sample we got from above
            inputx = torch.cat((inputx, indext_next), dim=1) # (Batch, block+1)
        return inputx


model = BigramModel(vocab_size).to(device)

predictions, loss = model(x, y)
print(predictions.shape)
print(loss)

print(decoding(model.generate(inputx = torch.zeros((1, 1), dtype=torch.long, device=device), max=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.8637, device='cuda:0', grad_fn=<NllLossBackward0>)

H dxyEB:llgmozIk Kxl!;sg.jLtlidGxs$I:y-H$Gd!nRj
IBxK'!-E&mvgtHzgb.B;ClOJ:xHKx?mLTuCS.?cbNb&k,Ov
kiy-


## Training

### create optimizer

In [21]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3)

In [22]:


for i in range (100000):

    #makes batches
    x, y = batchMaker('training', batch, block)

    #evaluate
    predictions , loss = model (x, y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


print(loss.item())



2.426051616668701


In [23]:
print(decoding(model.generate(inputx = torch.zeros((1, 1), dtype=torch.long , device=device), max=500)[0].tolist()))


Th phes:

BEN ascofo asu? s n mu kies pryouron iou er arouthe pred blle prethinol of ayse fithe cirathou whathat?-s
Whelldacttlyo lovea hoooorvepeaney uthy thur an belavere t I od, ame. d wifes his me to candeense cchananden utorgidelit ms hichan fof liso!

TE hafow
S:
NChary akeid harserou d
BY:
Cowag, s.
ARI tcckfetenoy s wil.
Isincathaconirero Corvarendlds s n cesir'

To, tre muro vies r ot
et wh, Andr uroindo jo and sh hed, dacert ins,
Yodwommns I s woveasountatorathe?

wed'sau hindeshlligin


## Now lets get our model ready for GPU

### Hyper Parameters

In [24]:

# number of sequences we will process


# maxcontent length for prediction
batch = 32 # how many independent sequences will we process in parallel?
block = 8 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

In [25]:

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = batchMaker(split, batch, block)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


model = BigramModel(vocab_size)
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: training loss {losses['train']:.4f}, validation loss {losses['val']:.4f}")

    # sample a batch of data
    x, y = batchMaker('training', batch, block)

    # evaluate the loss
    logits, loss = model(x, y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decoding(m.generate(context, max=500)[0].tolist()))

step 0: training loss 4.5967, validation loss 4.6049
step 300: training loss 2.7991, validation loss 2.8093
step 600: training loss 2.5623, validation loss 2.5653
step 900: training loss 2.5302, validation loss 2.5236
step 1200: training loss 2.5069, validation loss 2.5008
step 1500: training loss 2.4992, validation loss 2.5003
step 1800: training loss 2.4987, validation loss 2.4887
step 2100: training loss 2.5004, validation loss 2.4915
step 2400: training loss 2.4983, validation loss 2.4905
step 2700: training loss 2.4990, validation loss 2.4920
step 3000: training loss 2.4929, validation loss 2.4759
step 3300: training loss 2.4971, validation loss 2.4891
step 3600: training loss 2.4972, validation loss 2.4966
step 3900: training loss 2.4949, validation loss 2.4883
step 4200: training loss 2.4965, validation loss 2.4921
step 4500: training loss 2.4868, validation loss 2.4878
step 4800: training loss 2.4938, validation loss 2.4862
step 5100: training loss 2.4878, validation loss 2.486

## Make Changes to make the tranformer

### Hyper parameters

In [26]:
bath= 64 # how many independent sequences will we process in parallel?
block = 256 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 200
numberOfEmbeddings = 384
NumberofHeads = 6
numberOfLayers = 6
dropout = 0.2

## Coding the seld-attention head

In [27]:
class OneHead(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(numberOfEmbeddings, head_size, bias=False)
        self.query = nn.Linear(numberOfEmbeddings, head_size, bias=False)
        self.value = nn.Linear(numberOfEmbeddings, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block, block)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, block, vocab size)
        # output of size (batch, block, head size)
        Batch,Block,Vocab = x.shape
        k = self.key(x)   # (Batch,Block,head)
        q = self.query(x) # (Batch,Batch,heas)

        # attention scores
        weight = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (Bath, Batch, hs) @ (Bath, hs, Batch) -> (Bath, batch_size, batch_size)
        weight = weight.masked_fill(self.tril[:Block, :Block] == 0, float('-inf')) # (Bath, Block, Block)
        weight = F.softmax(weight, dim=-1) # (Bath, Block, Block)
        weight = self.dropout(weight)
        # perform the weighted aggregation of the values
        v = self.value(x) # (Bath,Block,hs)
        out = weight @ v # (Bath, Block, Block) @ (Bath, Block, hs) -> (Bath, Block, hs)
        return out



## Multi Head Attention

In [28]:
class MultiAttention(nn.Module):
    """ multiple heads of self-attention  """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([OneHead(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, numberOfEmbeddings)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out




## Forward Function

In [29]:
class FeedFoward(nn.Module):
    """ linear layerand a non-linear after """

    def __init__(self, numberOfEmbeddings):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(numberOfEmbeddings, 4 * numberOfEmbeddings),
            nn.ReLU(),
            nn.Linear(4 * numberOfEmbeddings, numberOfEmbeddings),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


In [30]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, numberOfEmbeddings, NumberofHeads):
        # numberOfEmbeddings: embedding dimension, NumberofHeads: the number of heads we'd like
        super().__init__()
        head_size = numberOfEmbeddings // NumberofHeads
        self.sa = MultiAttention(NumberofHeads, head_size)
        self.ffwd = FeedFoward(numberOfEmbeddings)
        self.ln1 = nn.LayerNorm(numberOfEmbeddings)
        self.ln2 = nn.LayerNorm(numberOfEmbeddings)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x



## Making the Chat Bot

In [34]:
class GhatBotModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, numberOfEmbeddings)
        self.position_embedding_table = nn.Embedding(block, numberOfEmbeddings)
        self.blocks = nn.Sequential(*[Block(numberOfEmbeddings, NumberofHeads) for _ in range(NumberofHeads)])
        self.ln_f = nn.LayerNorm(numberOfEmbeddings) # final layer norm
        self.lm_head = nn.Linear(numberOfEmbeddings, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        Batch, Blocks = idx.shape

        # idx and targets are both (Batch,Blocks) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (Batch,Blocks,vocab_size)
        pos_emb = self.position_embedding_table(torch.arange(Blocks, device=device)) # (T,C)
        x = tok_emb + pos_emb # (Batch,Blocks,vocab_size)
        x = self.blocks(x) # (Batch,Blocks,vocab_size)
        x = self.ln_f(x) # (Batch,Blocks,vocab_size)
        logits = self.lm_head(x) # (Batch,Blocks,vocab_size)

        if targets is None:
            loss = None
        else:
            Batch, Blocks, Vocabs = logits.shape
            logits = logits.view(Batch*Blocks, Vocabs)
            targets = targets.view(Batch*Blocks)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [35]:
model = GhatBotModel()
m = model.to(device)

print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

#PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: training loss {losses['train']:.4f}, validation loss {losses['val']:.4f}")

    # sample batch
    x, y = batchMaker('training', batch, block)

    # evaluate the loss
    logits, loss = model(x, y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()



10.788929 M parameters
step 0: training loss 4.1690, validation loss 4.1698
step 500: training loss 2.0377, validation loss 2.0380
step 1000: training loss 1.7176, validation loss 1.7210
step 1500: training loss 1.6135, validation loss 1.6116
step 2000: training loss 1.5469, validation loss 1.5446
step 2500: training loss 1.5163, validation loss 1.5169
step 3000: training loss 1.5069, validation loss 1.5055
step 3500: training loss 1.4943, validation loss 1.4956
step 4000: training loss 1.4829, validation loss 1.4806
step 4500: training loss 1.4819, validation loss 1.4773
step 5000: training loss 1.4869, validation loss 1.4905
step 5500: training loss 1.4857, validation loss 1.4925
step 6000: training loss 1.5092, validation loss 1.5056
step 6500: training loss 1.5173, validation loss 1.5219
step 7000: training loss 1.5345, validation loss 1.5232
step 7500: training loss 1.5483, validation loss 1.5424
step 8000: training loss 1.5534, validation loss 1.5574
step 8500: training loss 1.57

In [36]:

# generate from the model
texts = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decoding(m.generate(texts, max_new_tokens=500)[0].tolist()))


GLOUCESTER:
And hand, as we are deare, too we to him:
And, in dear some confess of nature's eye
Where Oxford had beshew some soften to the noise;
For yet would make it excellence again: the vant
I have had charged, such a cient many to tean
To flate peace in his place.

GLOUCESTER:
Part, look that we more harsh our breaths.
Take up within cur are: therefore I thence came;
Assist us, Couroy abound will he last.

BONA:
Indeed, I fear I have disness'd me with a pract,
To the entreat the like as wor
